### 导入需要使用的库

In [1]:
import pandas as pd
from pyecharts import GeoLines,Style

### 1.航班数据预处理

#### 航班数据来源

- 航班数据是通过一个开源的数据库获取的，网址是：https://openflights.org/data.html
- 下载其中的airline和airport数据即可
- 课程提供了下载好的数据，使用时，挑选自己感兴趣的airport和airline即可

In [2]:
# 导入机场和航线数据
routes = pd.read_excel('./data/航线数据.xlsx',sheetname='routes')
airports = pd.read_excel('./data/航线数据.xlsx',sheetname='airports')\
           [['Airport ID', 'City', 'Country', 'IATA', 'Latitude','Longitude', 'Altitude']]

In [3]:
routes.head(2)

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Equipment
0,2B,410,AER,2965,KZN,2990,CR2
1,2B,410,ASF,2966,KZN,2990,CR2


In [4]:
airports.head(2)

,Airport ID,City,Country,IATA,Latitude,Longitude,Altitude
0,1,Goroka,Papua New Guinea,GKA,-6.08169,145.391998,5282
1,2,Madang,Papua New Guinea,MAG,-5.20708,145.789001,20


#### 将机场的经纬度数据与机场城市对应的名称处理成dict类型

In [26]:
airports_geo = airports[['City','Latitude','Longitude']]    # 提取所需字段
airports_geo.index = airports_geo['City']                   # 将数据集的index设置为City
airports_geo = airports_geo[['Longitude','Latitude']]       # 剔除City 字段，因为其已经在索引中存在了
airports_geo = airports_geo.T.to_dict(orient='list')        # 对数据集进行行列转置，然后转换成字典类型

c:\programdata\anaconda3\lib\site-packages\pandas\core\frame.py:891: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  "columns will be omitted.", UserWarning)


In [27]:
airports_geo

{'Goroka': [145.391998291, -6.08168983459],
 'Madang': [145.789001465, -5.20707988739],
 'Mount Hagen': [144.296005249023, -5.82678985595703],
 'Nadzab': [146.725977, -6.569803],
 'Port Moresby': [147.220001220703, -9.44338035583496],
 'Wewak': [143.669006348, -3.58383011818],
 'Narssarssuaq': [-45.4259986877, 61.1604995728],
 'Godthaab': [-51.6781005859, 64.19090271],
 'Sondrestrom': [-50.7116031647, 67.0122218992],
 'Thule': [-68.7032012939, 76.5311965942],
 'Akureyri': [-18.0727005004882, 65.6600036621093],
 'Egilsstadir': [-14.4013996124267, 65.2833023071289],
 'Hofn': [-15.2272, 64.295601],
 'Husavik': [-17.426001, 65.952301],
 'Isafjordur': [-23.1352996826171, 66.0580978393554],
 'Keflavik': [-22.605600357056, 63.985000610352],
 'Patreksfjordur': [-23.965, 65.555801],
 'Reykjavik': [-21.9405994415, 64.1299972534],
 'Siglufjordur': [-18.9167, 66.133301],
 'Vestmannaeyjar': [-20.2789001464843, 63.4243011474609],
 'Sault Sainte Marie': [-84.5093994140625, 46.4850006103515],
 'Winnip

####  筛选起飞或者降落在北京的航班

In [12]:
# 查看北京机场信息
airports[airports["City"] == "Beijing"]

,Airport ID,City,Country,IATA,Latitude,Longitude,Altitude
3158,3364,Beijing,China,PEK,40.080101,116.584999,116
4945,6341,Beijing,China,NAY,39.782799,116.388000,0


In [13]:
airportID = airports[airports["City"] == "Beijing"]['Airport ID'].tolist()

In [14]:
airlines = routes[(routes["Source airport ID"].isin(airportID))|(routes["Destination airport ID"].isin(airportID))]
len(airlines)

1229

In [15]:
airlines.head()

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Equipment
634,3U,4608,CKG,3393,PEK,3364,321 319 320
689,3U,4608,CTU,3395,PEK,3364,330 321 320
859,3U,4608,PEK,3364,CKG,3393,319 321 320
860,3U,4608,PEK,3364,CTU,3395,321 320 330
861,3U,4608,PEK,3364,WXN,6403,320


### 2.1 北京起飞航班

In [16]:
source = airlines[airlines['Source airport ID'].isin(airportID)][['Source airport ID','Destination airport ID']].drop_duplicates()
source["Source airport"] = "Beijing"
source = pd.merge(source,airports,left_on="Destination airport ID",right_on="Airport ID",how = "left")[["Source airport","City"]]

In [22]:
airline_source = source.values.tolist()

In [28]:
style = Style(
    title_color="#fff",
    title_pos = "center",
    width=980,
    height=600,
    background_color="#404a59"
)

style_geo = style.add(
    maptype="world",
    is_label_show=False,
    line_curve=0.1,
    line_opacity=0.6,
    legend_text_color="#eee",
    legend_pos="right",
    geo_effect_symbol="plane",
    geo_effect_symbolsize=15,
    label_color=['#a6c84c', '#ffa022', '#46bee9'],
    label_pos="right",
    label_formatter="{b}",
    label_text_color="#eee",
    geo_cities_coords = airports_geo
)

geolines = GeoLines("北京起飞航班的航线", **style.init_style)
geolines.add(name = "",data = airline_source,is_legend_show=False,**style_geo)
geolines.render('./html/airline_source.html')
geolines

### 2.2 北京降落航班

In [29]:
destination = airlines[airlines['Destination airport ID'].isin(airportID)][['Source airport ID','Destination airport ID']].drop_duplicates()
destination["Destination airport"] = "Beijing"
destination = pd.merge(destination,airports,left_on="Source airport ID",right_on="Airport ID",how = "left")[["City","Destination airport"]]

In [30]:
airline_destination = destination.values.tolist()

In [39]:
style = Style(
    title_color="#fff",
    title_pos = "center",
    width=1510,
    height=840,
    background_color="#404a59"
)

style_geo = style.add(
    maptype="world",
    is_label_show=False,
    line_curve=0.1,
    line_opacity=0.6,
    legend_text_color="#eee",
    legend_pos="right",
    geo_effect_symbol="plane",
    symbol_size=5,
    geo_effect_symbolsize=15,
    label_color=['#a6c84c',"red"],
    label_pos="right",
    label_formatter="{b}",
    label_text_color="#eee",
    geo_cities_coords=airports_geo
)

geolines = GeoLines("航班数据", **style.init_style)
geolines.add(name = "降落",data = airline_destination,is_legend_show=False,**style_geo)
geolines.add(name = "起飞",data = airline_source,is_legend_show=True,**style_geo)
geolines.render('./html/airline_des.html')
geolines

### 3.绘制任意城市起降航班航线